<a href="https://colab.research.google.com/github/smitdesai1010/COVID-19-Detection/blob/master/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}'}

In [6]:
import json

token = {"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}
with open('/content/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v /content
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset


import zipfile

zf = zipfile.ZipFile('/content/datasets/amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset/covid19-pneumonia-normal-chest-xray-pa-dataset.zip') 
zf.extractall()

!rm -rf 'datasets' #Deleting the the downloaded Zip files after extracting it 

- path is now set to: /content
 99% 1.90G/1.91G [00:58<00:00, 47.6MB/s]
100% 1.91G/1.91G [00:59<00:00, 34.8MB/s]


In [7]:
import tensorflow as tf
import cv2
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

In [8]:
def create_dataset(name):
  org_path = '/content/'+ name
  train_path = '/content/train/'+ name
  val_path = '/content/validation/'+ name
  test_path = '/content/test/'+ name

  if not os.path.exists(train_path):
    os.mkdir(train_path)
    os.mkdir(val_path)
    os.mkdir(test_path)

  path, dirs, files = next(os.walk(org_path))
  file_count = len(files)
  i = 0  
    
  for file in os.listdir(org_path):
    
    if not os.path.isfile(os.path.join(org_path,file)):
      continue

    if i < file_count*0.75:
     shutil.copy2(os.path.join(org_path,file), os.path.join(train_path,file))
    
    elif i > file_count*0.9:
     shutil.copy2(os.path.join(org_path,file), os.path.join(test_path,file))

    else:
     shutil.copy2(os.path.join(org_path,file), os.path.join(val_path,file))
     
    i = i + 1 

https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71

In [9]:
if not os.path.exists('/content/train'):
  !mkdir /content/train
  !mkdir /content/validation
  !mkdir /content/test
 
create_dataset('covid')
create_dataset('normal')
create_dataset('pneumonia')

!rm -rf 'covid'
!rm -rf 'normal'
!rm -rf 'pneumonia'

In [10]:
train_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

val_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

test_generator = ImageDataGenerator(
    rescale = 1. / 255 ,
    horizontal_flip = True
)

train_data = train_generator.flow_from_directory(
    '/content/train' ,
    target_size = (1024,1024) ,
    batch_size = 32 ,
)

val_data = val_generator.flow_from_directory(
    '/content/validation' ,
    target_size = (1024,1024) ,
    batch_size = 32 ,
)

test_data = test_generator.flow_from_directory(
    '/content/test' ,
    target_size = (1024,1024) ,
    batch_size = 32 ,
)

Found 5173 images belonging to 3 classes.
Found 1038 images belonging to 3 classes.
Found 691 images belonging to 3 classes.


In [13]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, 1024, 1024)
else:
    input_shape = (1024, 1024, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_data ,
    steps_per_epoch = 5178/32 ,
    epochs = 5 ,
    validation_data=val_data ,
    validation_steps = 1032/32 ,
    verbose = 2 ,
    use_multiprocessing = False ,
    workers = 1,
)